## Kasra Samadi 993623030

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

### reading weatherHistory_Train and weatherHistory_Test from csv

In [2]:
df_train = pd.read_csv("Datasets\weatherHistory\weatherHistory_Train.csv")
df_train = pd.DataFrame(df_train)
df_test = pd.read_csv("Datasets\weatherHistory\weatherHistory_Test.csv")
df_test = pd.DataFrame(df_test)
df_train.head()

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [3]:
df_train.isnull()
df_train.isna().sum()

Formatted Date                0
Summary                       0
Precip Type                 143
Temperature (C)               0
Apparent Temperature (C)      0
Humidity                      0
Wind Speed (km/h)             0
Wind Bearing (degrees)        0
Visibility (km)               0
Loud Cover                    0
Pressure (millibars)          0
Daily Summary                 0
dtype: int64

In [4]:
df_test.isnull()
df_test.isna().sum()

Formatted Date              0
Summary                     0
Precip Type               374
Temperature (C)             0
Humidity                    0
Wind Speed (km/h)           0
Wind Bearing (degrees)      0
Visibility (km)             0
Loud Cover                  0
Pressure (millibars)        0
Daily Summary               0
dtype: int64

### convert Formatted Date to numerical calues

In [5]:
import datetime as dt
df_train['Formatted Date'] = pd.to_datetime(df_train['Formatted Date'])
df_train['Formatted Date']=df_train['Formatted Date'].map(dt.datetime.toordinal)

df_test['Formatted Date'] = pd.to_datetime(df_test['Formatted Date'])
df_test['Formatted Date']=df_test['Formatted Date'].map(dt.datetime.toordinal)



In [6]:
df_train

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,732402,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,732402,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,732402,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,732402,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,732402,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
...,...,...,...,...,...,...,...,...,...,...,...,...
67512,735008,Mostly Cloudy,rain,17.594444,17.594444,0.84,10.6421,279.0,9.9820,0.0,1010.25,Partly cloudy throughout the day.
67513,735008,Partly Cloudy,rain,17.566667,17.566667,0.80,13.5723,269.0,11.0285,0.0,1011.30,Partly cloudy throughout the day.
67514,735008,Partly Cloudy,rain,17.000000,17.000000,0.81,14.0392,260.0,15.8263,0.0,1011.82,Partly cloudy throughout the day.
67515,735008,Partly Cloudy,rain,15.355556,15.355556,0.78,14.0231,250.0,15.8263,0.0,1011.83,Partly cloudy throughout the day.


In [7]:
df_train["Precip Type"].value_counts()
df_test["Precip Type"].isna().sum()
df_test["Precip Type"].value_counts()
df_train["Precip Type"]

0        rain
1        rain
2        rain
3        rain
4        rain
         ... 
67512    rain
67513    rain
67514    rain
67515    rain
67516    rain
Name: Precip Type, Length: 67517, dtype: object

### drop null values and replace with median of Precip Type, then convert Precip Type to numerical values

In [8]:
temp_Xtrain=df_test["Precip Type"].dropna()
temp_Xtest=df_test["Precip Type"].dropna()

mode_Xtrain = temp_Xtrain.mode()
mode_Xtest = temp_Xtest.mode()

df_train["Precip Type"].fillna(mode_Xtrain[0], inplace=True)
df_test["Precip Type"].fillna(mode_Xtest[0], inplace=True)

Precip_Type = {'snow': 2,'rain': 1} 
df_train["Precip Type"] = [Precip_Type[item] for item in df_train["Precip Type"]]
df_test["Precip Type"] = [Precip_Type[item] for item in df_test["Precip Type"]]


### define x_train without Summary , Daily Summary and Apparent Temperature (C) columns AND define x_test without Summary , Daily Summary columns

In [9]:
x_train = df_train.drop(["Summary","Daily Summary","Apparent Temperature (C)"], axis=1)
y_train = pd.DataFrame(df_train["Apparent Temperature (C)"])
x_test = df_test.drop(["Summary","Daily Summary"], axis=1)

In [10]:
x_train

,Formatted Date,Precip Type,Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars)
0,732402,1,9.472222,0.89,14.1197,251.0,15.8263,0.0,1015.13
1,732402,1,9.355556,0.86,14.2646,259.0,15.8263,0.0,1015.63
2,732402,1,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94
3,732402,1,8.288889,0.83,14.1036,269.0,15.8263,0.0,1016.41
4,732402,1,8.755556,0.83,11.0446,259.0,15.8263,0.0,1016.51
...,...,...,...,...,...,...,...,...,...
67512,735008,1,17.594444,0.84,10.6421,279.0,9.9820,0.0,1010.25
67513,735008,1,17.566667,0.80,13.5723,269.0,11.0285,0.0,1011.30
67514,735008,1,17.000000,0.81,14.0392,260.0,15.8263,0.0,1011.82
67515,735008,1,15.355556,0.78,14.0231,250.0,15.8263,0.0,1011.83


### normalize x_train and x_test with Z Score

In [11]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
x_train=pd.DataFrame(poly.fit_transform(x_train))
poly_test = PolynomialFeatures(2)
x_test=pd.DataFrame(poly_test.fit_transform(x_test))

In [12]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,1.0,732402.0,1.0,9.472222,0.89,14.1197,251.0,15.8263,0.0,1015.13,...,63001.0,3972.4013,0.0,254797.63,250.471772,0.0,16065.751919,0.0,0.0,1.030489e+06
1,1.0,732402.0,1.0,9.355556,0.86,14.2646,259.0,15.8263,0.0,1015.63,...,67081.0,4099.0117,0.0,263048.17,250.471772,0.0,16073.665069,0.0,0.0,1.031504e+06
2,1.0,732402.0,1.0,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,...,41616.0,3051.2076,0.0,207251.76,223.708858,0.0,15195.312986,0.0,0.0,1.032134e+06
3,1.0,732402.0,1.0,8.288889,0.83,14.1036,269.0,15.8263,0.0,1016.41,...,72361.0,4257.2747,0.0,273414.29,250.471772,0.0,16086.009583,0.0,0.0,1.033089e+06
4,1.0,732402.0,1.0,8.755556,0.83,11.0446,259.0,15.8263,0.0,1016.51,...,67081.0,4099.0117,0.0,263276.09,250.471772,0.0,16087.592213,0.0,0.0,1.033293e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67512,1.0,735008.0,1.0,17.594444,0.84,10.6421,279.0,9.9820,0.0,1010.25,...,77841.0,2784.9780,0.0,281859.75,99.640324,0.0,10084.315500,0.0,0.0,1.020605e+06
67513,1.0,735008.0,1.0,17.566667,0.80,13.5723,269.0,11.0285,0.0,1011.30,...,72361.0,2966.6665,0.0,272039.70,121.627812,0.0,11153.122050,0.0,0.0,1.022728e+06
67514,1.0,735008.0,1.0,17.000000,0.81,14.0392,260.0,15.8263,0.0,1011.82,...,67600.0,4114.8380,0.0,263073.20,250.471772,0.0,16013.366866,0.0,0.0,1.023780e+06
67515,1.0,735008.0,1.0,15.355556,0.78,14.0231,250.0,15.8263,0.0,1011.83,...,62500.0,3956.5750,0.0,252957.50,250.471772,0.0,16013.525129,0.0,0.0,1.023800e+06


In [13]:
x_test

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,1.0,735008.0,1.0,13.800000,0.77,12.3809,211.0,11.2056,0.0,1013.92,...,44521.0,2364.3816,0.0,213937.12,125.565471,0.0,11361.581952,0.0,0.0,1.028034e+06
1,1.0,735008.0,1.0,14.933333,0.72,11.2056,230.0,11.2056,0.0,1014.46,...,52900.0,2577.2880,0.0,233325.80,125.565471,0.0,11367.632976,0.0,0.0,1.029129e+06
2,1.0,735008.0,1.0,16.872222,0.64,14.0714,233.0,10.3523,0.0,1014.91,...,54289.0,2412.0859,0.0,236474.03,107.170115,0.0,10506.652793,0.0,0.0,1.030042e+06
3,1.0,735008.0,1.0,17.800000,0.64,12.8800,233.0,10.0464,0.0,1015.25,...,54289.0,2340.8112,0.0,236553.25,100.930153,0.0,10199.607600,0.0,0.0,1.030733e+06
4,1.0,735008.0,1.0,18.933333,0.56,20.3987,250.0,11.2700,0.0,1015.26,...,62500.0,2817.5000,0.0,253815.00,127.012900,0.0,11441.980200,0.0,0.0,1.030753e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28931,1.0,736216.0,1.0,26.016667,0.43,10.9963,31.0,16.1000,0.0,1014.36,...,961.0,499.1000,0.0,31445.16,259.210000,0.0,16331.196000,0.0,0.0,1.028926e+06
28932,1.0,736216.0,1.0,24.583333,0.48,10.0947,20.0,15.5526,0.0,1015.16,...,400.0,311.0520,0.0,20303.20,241.883367,0.0,15788.377416,0.0,0.0,1.030550e+06
28933,1.0,736216.0,1.0,22.038889,0.56,8.9838,30.0,16.1000,0.0,1015.66,...,900.0,483.0000,0.0,30469.80,259.210000,0.0,16352.126000,0.0,0.0,1.031565e+06
28934,1.0,736216.0,1.0,21.522222,0.60,10.5294,20.0,16.1000,0.0,1015.95,...,400.0,322.0000,0.0,20319.00,259.210000,0.0,16356.795000,0.0,0.0,1.032154e+06


### Linear Regression

In [14]:
regr = linear_model.LinearRegression()
regr.fit (x_train, y_train)
print ('Coefficients: ', regr.coef_)

Coefficients:  [[-1.06095612e-04 -2.38685692e-02 -4.92628263e-01 -2.68355912e-02
  -1.71247399e+01  2.79385630e-01 -1.29788925e-01  4.06263750e+00
   3.91351553e-06 -1.76203084e-02  1.62501537e-08  5.47047220e-06
   1.75963234e-06  2.00601379e-05 -8.83945129e-07  1.79915090e-07
  -5.76972086e-06  2.57285365e-08  2.30353479e-08 -1.47232741e+00
  -1.97776207e-01  1.67558910e+00 -4.12484433e-02 -3.99287813e-04
   3.51600029e-02 -2.66303261e-10 -9.60197856e-05 -5.48389373e-03
   9.51805201e-02  1.16761656e-02 -1.77262075e-05  2.31308092e-03
  -7.46664675e-11  1.84007815e-07 -1.38713380e+00  8.43732735e-02
   1.80699448e-04  3.02989292e-02  1.39444012e-13 -2.11953382e-05
   3.13488409e-03 -7.01705949e-05  4.78563760e-03  0.00000000e+00
  -6.45476603e-06 -9.62776403e-07  1.97879614e-06  0.00000000e+00
  -4.92110618e-07  3.37439302e-04  0.00000000e+00  1.48430007e-05
   0.00000000e+00  0.00000000e+00  8.73010861e-07]]


### predict y_pred

In [15]:
y_pred = regr.predict(x_test)
y_pred = pd.DataFrame(y_pred,columns=['Apparent Temperature (C)'])
y_pred.head(20)

,Apparent Temperature (C)
0,13.280894
1,14.670674
2,16.659482
3,17.747942
4,18.858438
5,19.999411
6,21.194385
7,22.313878
8,22.670632
9,22.748947


### save to csv

In [16]:
y_pred.to_csv("weather_pred.csv",index=False)